## Создать таблицу с движком MergeTree, вставить в неё данные и проверить, как данные разбиваются на части. 

In [50]:
import time
from clickhouse_driver import Client

In [51]:
ch = Client(
    host='localhost',
    port=9000,
    user='admin',
    password='admin'
)

In [52]:
ch.execute('''
CREATE TABLE IF NOT EXISTS default.test_mergetree
(
  id UInt64,
  event_date DateTime,
  description String
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(event_date)
ORDER BY id
SETTINGS index_granularity = 8192;
''')

[]

In [53]:
ch.execute("INSERT INTO default.test_mergetree VALUES \
           (1, '2025-08-01 12:00:00', 'test event 1'),\
           (2, '2025-08-01 12:30:00', 'test event 2');")

[]

что значит пример записи из system.parts: [('202508_1_1_0', '202508', 1)]

    '202508_1_1_0' — это имя части (part), под которым физически хранится блок данных на диске. Формат имени части несет информацию о партиции, номерах блоков и уровне слияния.

    '202508' — это значение партиции (partition). В данном случае партиция задействует формат YYYYMM (год и месяц) и соответствует августу 2025 года.

    '1' — уровень слияния части (level). Значение 0 означает, что часть is новая и еще не подвергалась слиянию, а 1 и выше — были объединения с другими частями.

Иными словами, таблица с движком MergeTree состоит из нескольких частей (parts), которые соответствуют разным временным партициям, и эти части проходят процесс слияния (merge), при котором уровень части увеличивается. 

In [54]:
ch.execute("SELECT name, partition, active FROM system.parts WHERE table = 'test_mergetree' AND database = 'default';")
# Это покажет, что каждая вставка создает свои части, которые потом сливаются фоновой задачей. 

[('202508_1_10_2', '202508', 1),
 ('202508_11_15_1', '202508', 1),
 ('202508_16_16_0', '202508', 1)]

## Сравнить производительность вставки в MergeTree и Log

In [55]:
ch.execute('''
CREATE TABLE IF NOT EXISTS default.test_log
(
  id UInt64,
  event_date DateTime,
  description String
)
ENGINE = Log;
''')

[]



    Вставка в таблицу с движком Log происходит быстрее, так как Log — это очень простой движок, который просто добавляет данные подряд без индексов и сортировки.

    Вставка в MergeTree медленнее, потому что там идет дополнительная работа:

        формируются части с данными,

        данные сортируются по ORDER BY,

        создаются индексы (index_granularity),

        а затем фоновые процессы сливают маленькие части в большие.

    С другой стороны, запросы на чтение и аналитика по таблице MergeTree гораздо эффективнее, потому что индексы помогают быстро находить нужные данные и пропускать неактуальные.

    Запросы по таблице Log требуют полной проверки всех данных, так как там нет индексов, поэтому чтение медленнее, больше затрат ресурсов.

Таким образом, Log движок оптимален для очень быстрых вставок, но плохо подходит для аналитики. MergeTree подходит для аналитических сценариев с большими данными, где важна быстрая выборка, несмотря на более сложную вставку. 

In [56]:
# Для MergeTree
start_time = time.time()

ch.execute("""
    INSERT INTO default.test_mergetree
    SELECT number AS id, now() AS event_date, toString(number) AS description FROM numbers(10000000);
""")

end_time = time.time()
elapsed = end_time - start_time
print(f"Время вставки данных в MergeTree: {elapsed} секунд")

Время вставки данных в MergeTree: 1.3696188926696777 секунд


In [57]:
# Для Log
start_time = time.time()

ch.execute("INSERT INTO default.test_log \
           SELECT number AS id, now() AS event_date, toString(number) AS description FROM numbers(10000000);")

end_time = time.time()
elapsed = end_time - start_time
print(f"Время вставки данных в Log: {elapsed} секунд")

Время вставки данных в Log: 0.947180986404419 секунд


## Создать таблицу с ReplacingMergeTree, вставить дублирующиеся записи с разными версиями и проверить, как движок оставляет только последние версии. 

In [58]:
ch.execute('''
CREATE TABLE IF NOT EXISTS default.test_replacing
(
  id UInt64,
  version UInt64,
  description String
)
ENGINE = ReplacingMergeTree(version)
ORDER BY id;
''')

[]

In [59]:
ch.execute("INSERT INTO default.test_replacing VALUES \
           (1, 1, 'desc v1'), \
           (1, 2, 'desc v2'), \
           (2, 1, 'another desc');")

[]

In [60]:
ch.execute("SELECT * FROM default.test_replacing;")

[(1, 2, 'desc v2'),
 (2, 1, 'another desc'),
 (1, 2, 'desc v2'),
 (2, 1, 'another desc')]

## Проанализировать поведение фонового слияния (merge) частей в MergeTree с большим объемом данных. 

In [61]:
# После больших вставок: 
ch.execute("INSERT INTO default.test_mergetree SELECT number AS id, now() AS event_date, toString(number) AS description FROM numbers(1000000);")

[]

In [62]:
# Потом проверить в системной таблице parts: 
ch.execute("SELECT name, partition, active, rows, bytes_on_disk, modification_time FROM system.parts WHERE table = 'test_mergetree' ORDER BY modification_time DESC LIMIT 10;")

[('202508_26_26_0',
  '202508',
  1,
  1000000,
  8177057,
  datetime.datetime(2025, 8, 28, 18, 41, 17)),
 ('202508_16_21_1',
  '202508',
  1,
  5559767,
  45196099,
  datetime.datetime(2025, 8, 28, 18, 41, 16)),
 ('202508_25_25_0',
  '202508',
  1,
  1104376,
  8968204,
  datetime.datetime(2025, 8, 28, 18, 41, 15)),
 ('202508_24_24_0',
  '202508',
  1,
  1111953,
  9024557,
  datetime.datetime(2025, 8, 28, 18, 41, 15)),
 ('202508_23_23_0',
  '202508',
  1,
  1111953,
  9029691,
  datetime.datetime(2025, 8, 28, 18, 41, 15)),
 ('202508_21_21_0',
  '202508',
  0,
  1111953,
  9024627,
  datetime.datetime(2025, 8, 28, 18, 41, 15)),
 ('202508_20_20_0',
  '202508',
  0,
  1111953,
  9024862,
  datetime.datetime(2025, 8, 28, 18, 41, 15)),
 ('202508_22_22_0',
  '202508',
  1,
  1111953,
  9029038,
  datetime.datetime(2025, 8, 28, 18, 41, 15)),
 ('202508_17_17_0',
  '202508',
  0,
  1111953,
  9087639,
  datetime.datetime(2025, 8, 28, 18, 41, 15)),
 ('202508_18_18_0',
  '202508',
  0,
  111195

In [63]:
# Фоновые процессы сливают маленькие части (parts) в большие, повышая производительность чтения и уменьшая накладные расходы на индексы. Это исполняется автоматически, но можно запускать вручную: 
ch.execute("OPTIMIZE TABLE default.test_mergetree;")

[]

# Партиционирование 

Партиционирование в ClickHouse — это способ логически разделить большую таблицу на более мелкие независимые части (партиции) по определенному признаку. Это похоже на разбиение большого архива на папки по месяцам или годам для удобства поиска и управления. 

Представим, что есть таблица с данными о событиях, и у каждого события есть дата ( event_date). Можно разбить эти данные на партиции по месяцам — например, все события августа 2025 года — в одну папку, сентября 2025 — в другую и так далее.

Тогда при поиске событий в августе ClickHouse будет читать только «папку» с августовскими данными, а не всю таблицу целиком. Это сильно ускорит запросы. 

In [64]:
ch.execute('''
CREATE TABLE IF NOT EXISTS default.partitioned_events
(
    id UInt64,
    event_date DateTime,
    description String
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(event_date)
ORDER BY id;
''')

[]

In [65]:
ch.execute("INSERT INTO default.partitioned_events VALUES \
(1, '2025-08-10 12:00:00', 'event in Aug'), \
(2, '2025-09-05 15:30:00', 'event in Sep'), \
(3, '2025-08-20 09:00:00', 'another event Aug'), \
(4, '2025-07-25 17:45:00', 'event in Jul');")

[]

In [66]:
# Партиции хранятся на диске как отдельные папки. Их можно увидеть через системную таблицу parts: 
ch.execute('''
SELECT partition, name, active, rows, bytes_on_disk
FROM system.parts
WHERE table = 'partitioned_events' AND database = 'default'
ORDER BY partition;
''')
# Каждая уникальная партиция ( partition — например, 202507, 202508) отображается отдельно.

[('202507', '202507_3_3_0', 1, 1, 453),
 ('202507', '202507_6_6_0', 1, 1, 453),
 ('202508', '202508_1_1_0', 1, 2, 481),
 ('202508', '202508_4_4_0', 1, 2, 481),
 ('202509', '202509_2_2_0', 1, 1, 453),
 ('202509', '202509_5_5_0', 1, 1, 453)]

In [67]:
ch.execute('''
SELECT *
FROM default.partitioned_events
WHERE toYYYYMM(event_date) = 202508;
''')
# Для сравнения можно создать аналогичную таблицу без партиционирования, 
# вставить те же данные и выполнить такой же запрос, 
# чтобы посмотреть, что партиционирование ограничивает объем сканируемых данных. 

[(1, datetime.datetime(2025, 8, 10, 12, 0), 'event in Aug'),
 (3, datetime.datetime(2025, 8, 20, 9, 0), 'another event Aug'),
 (1, datetime.datetime(2025, 8, 10, 12, 0), 'event in Aug'),
 (3, datetime.datetime(2025, 8, 20, 9, 0), 'another event Aug')]

#### Эксперименты с разными выражениями в PARTITION BY

Партиционирование по неделям:
PARTITION BY toYYYYWW(event_date)  -- Год + номер недели

Партиционирование по региону (если есть столбец region): 
PARTITION BY region

Комбинированное партиционирование (например, регион + месяц): 
PARTITION BY (region, toYYYYMM(event_date))

# Ключи сортировки и ORDER BY

# Анализ влияния SAMPLE BY на хранение и распределение данных 

SAMPLE BY — это механизм в ClickHouse, который позволяет быстро выбирать случайную часть данных из большой таблицы для ускорения запросов с приемлемой точностью. 

Представим огромную таблицу с миллионами строк. Если нужно быстро получить приблизительный результат, то читать всю таблицу долго и дорого.

SAMPLE BY разбивает данные на равные части (гранулы) по ключу, и с ключевым словом SAMPLE в SELECT можно прочитать только часть этих гранул, например 10%. Это похоже на то, как если взять пробы из большой емкости, чтобы быстро оценить содержимое. 

In [69]:
ch.execute('''
CREATE TABLE IF NOT EXISTS default.events_sampled
(
    id UInt64,
    event_date DateTime,
    description String
)
ENGINE = MergeTree
ORDER BY (id, intHash32(id))
SAMPLE BY intHash32(id);  -- Ключ выборки для выборочного чтения данных
''')

[]

In [70]:
ch.execute("INSERT INTO default.events_sampled SELECT number, now(), toString(number) FROM numbers(1000000);")

[]

In [71]:
# Запрос на выборку 10% данных: 
ch.execute('''
SELECT count(*)
FROM default.events_sampled
SAMPLE 0.1;
''')

[(100490,)]

# Настройки производительности и индексы (index_granularity)